# Part IV: Modelling

## Basic settings

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
import pickle
import sys
import warnings
from datetime import datetime, timedelta

import lightgbm as lgbm
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import shap
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

warnings.filterwarnings("ignore")

# Set plotting style
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("deep")

# Set random seed for reproducibility
np.random.seed(2025)

d:\Asus\App\miniconda\workspace\envs\sales_forcast_xai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [3]:
src_path = os.path.abspath(os.path.join("../..", "src"))
if src_path not in sys.path:
    sys.path.append(src_path)

In [4]:
from utils.plots import plot_forecast_single
from utils.utils import flatten_prophet_predictions, weighted_absolute_percentage_error

In [5]:
import os
import pandas as pd
import numpy as np

DATA_DIR = "../../data"

# 1. LOAD DATA ĐÃ PREPROCESS VÀ FEATURE ENGINEERING
df_sales = pd.read_csv(
    os.path.join(DATA_DIR, "data_processed/sales_data_preprocessed.csv"),
    parse_dates=["date"]
)
df_weather = pd.read_csv(
    os.path.join(DATA_DIR, "data_processed/weather_preprocessed.csv"),
    parse_dates=["date"]
)
# df_weather_key_store_merged = pd.read_csv(
#     os.path.join(DATA_DIR, "data_processed/weather_key_store_merged.csv"),
#     parse_dates=["date"]
# )

# Đây là file đã có is_kaggle_test và toàn bộ features
df_features = pd.read_feather(os.path.join(DATA_DIR,'data_processed/feature_engineered_data_89_features.feather'))

print("Full feature data:", df_features.shape)
print("Kaggle test rows:", df_features['is_kaggle_test'].sum())
print("Train rows:", (df_features['is_kaggle_test'] == 0).sum())

Full feature data: (686187, 89)
Kaggle test rows: 526917
Train rows: 159270


## Load data

In [6]:
df_features['is_valid'] = 0
mask_train = df_features['is_kaggle_test'] == 0
cutoff_date = pd.Timestamp("2014-08-01")
df_features.loc[mask_train & (df_features['date'] >= cutoff_date), 'is_valid'] = 1

# 2. Tách train/valid và kaggle test
df_train = df_features[(df_features['is_kaggle_test'] == 0) & (df_features['is_valid'] == 0)].copy()
df_valid = df_features[(df_features['is_kaggle_test'] == 0) & (df_features['is_valid'] == 1)].copy()
df_kaggle_test = df_features[df_features['is_kaggle_test'] == 1].copy()

print("Final splits:")
print("  Train:", df_train.shape)
print("  Valid:", df_valid.shape)
print("  Kaggle test:", df_kaggle_test.shape)

Final splits:
  Train: (153496, 90)
  Valid: (5774, 90)
  Kaggle test: (526917, 90)


In [7]:
df_kaggle_test.head()

,date,store_nbr,item_nbr,units,logunits,is_kaggle_test,station_nbr,tmax,depart,cool,...,logunits_ewma_14d_a05,logunits_ewma_28d_a05,logunits_ewma_7d_a075,logunits_ewma_14d_a075,logunits_ewma_28d_a075,store_sum_7d,store_mean_7d,item_sum_7d,item_mean_7d,is_valid
159270,2013-04-01,2,1,NaN,NaN,True,14,71.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,8.670772,1.238682,7.203406,1.029058,0
159271,2013-04-01,3,1,NaN,NaN,True,7,68.0,6.2,0.0,...,NaN,NaN,NaN,NaN,NaN,8.825560,1.260794,6.510258,1.085043,0
159272,2013-04-01,6,1,NaN,NaN,True,14,71.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,12.102488,1.728927,5.817111,1.163422,0
159273,2013-04-01,7,1,NaN,NaN,True,6,86.0,6.0,5.0,...,NaN,NaN,NaN,NaN,NaN,8.648221,1.235460,5.123964,1.280991,0
159274,2013-04-01,8,1,NaN,NaN,True,4,87.0,8.0,9.0,...,NaN,NaN,NaN,NaN,NaN,9.572480,1.367497,3.178054,1.059351,0


In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 153496 entries, 0 to 159249
Data columns (total 90 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   date                    153496 non-null  datetime64[ns]
 1   store_nbr               153496 non-null  int64         
 2   item_nbr                153496 non-null  int64         
 3   units                   153496 non-null  float64       
 4   logunits                153496 non-null  float64       
 5   is_kaggle_test          153496 non-null  bool          
 6   station_nbr             153496 non-null  int64         
 7   tmax                    153496 non-null  float64       
 8   depart                  153496 non-null  float64       
 9   cool                    153496 non-null  float64       
 10  sunrise                 153496 non-null  float64       
 11  sunset                  153496 non-null  float64       
 12  snowfall                153496 non-

## Build lightgbm model

In [9]:
drop_cols = [
    'date', 'units', 'logunits',
    'is_kaggle_test', 'is_valid',
    'store_nbr', 'item_nbr', 'station_nbr',
    'tmax', 'depart', 'cool', 'sunrise', 'sunset',
    'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
    'resultspeed', 'resultdir',
    'BCFG', 'BLDU', 'BLSN', 'BR', 'DU', 'DZ', 'FG', 'FG+',
    'FU', 'FZDZ', 'FZFG', 'FZRA', 'GR', 'GS', 'HZ', 'MIFG',
    'PL', 'PRFG', 'RA', 'SG', 'SN', 'SQ', 'TS', 'TSRA',
    'TSSN', 'UP', 'VCFG', 'VCTS'
]

feature_cols = [c for c in df_features.columns if c not in drop_cols]

### Build a lightgbm model

In [ ]:
def create_lightgbm_model(df_train, df_valid, feature_cols):
    print("Building LightGBM models per (store, item)...")

    pairs = df_train[['store_nbr', 'item_nbr']].drop_duplicates()

    lgbm_models = {}
    lgbm_metrics = []
    all_actual, all_pred = [], []

    for _, row in pairs.iterrows():
        s, i = int(row['store_nbr']), int(row['item_nbr'])

        train_si = df_train[(df_train['store_nbr'] == s) &
                            (df_train['item_nbr'] == i)]
        valid_si = df_valid[(df_valid['store_nbr'] == s) &
                            (df_valid['item_nbr'] == i)]

        # Không đủ dữ liệu thì bỏ (giống Prophet)
        if len(train_si) < 20:
            continue

        X_tr = train_si[feature_cols]
        y_tr = train_si['logunits']
        X_va = valid_si[feature_cols]
        y_va = valid_si['logunits']

        # Sử dụng tham số tối ưu từ Optuna
        model = lgbm.LGBMRegressor(
            objective="regression",
            metric="rmse",
            boosting_type="gbdt",
            verbosity=-1,
            num_leaves=127,
            learning_rate=0.012528590527215604,
            feature_fraction=0.8044026085648729,
            bagging_fraction=0.9028865865109351,
            bagging_freq=7,
            min_child_samples=42,
            lambda_l1=7.327783159306205e-08,
            lambda_l2=0.0035687388325977474,
            max_depth=11,
            n_estimators=2000,
        )
        
        # Train với early stopping nếu có validation set
        if not valid_si.empty:
            model.fit(
                X_tr, y_tr,
                eval_set=[(X_va, y_va)],
                callbacks=[
                    lgbm.early_stopping(stopping_rounds=100),
                    lgbm.log_evaluation(0)
                ]
            )
        else:
            model.fit(X_tr, y_tr)

        key = (s, i)
        lgbm_models[key] = model

        if not valid_si.empty:
            preds = model.predict(X_va)
            mae = mean_absolute_error(y_va, preds)
            rmse = np.sqrt(mean_squared_error(y_va, preds))
            wape = weighted_absolute_percentage_error(y_va, preds)
            lgbm_metrics.append((s, i, mae, rmse, wape))
            all_actual.extend(y_va)
            all_pred.extend(preds)

    if lgbm_metrics:
        mae_avg = np.mean([m[2] for m in lgbm_metrics])
        rmse_avg = np.mean([m[3] for m in lgbm_metrics])
        wape_avg = weighted_absolute_percentage_error(
            np.array(all_actual), np.array(all_pred)
        )
        print(f"\nOverall LGBM per-(store,item) valid metrics:")
        print(f"MAE: {mae_avg:.4f} | RMSE: {rmse_avg:.4f} | WAPE: {wape_avg:.4f}")

    print("Models built:", len(lgbm_models))
    return lgbm_models, lgbm_metrics

In [11]:
lgbm_models, lgbm_metrics = create_lightgbm_model(
    df_train, df_valid, feature_cols
)

Building LightGBM models per (store, item) with 5-fold CV...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[59]	valid_0's rmse: 0.695935
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[336]	valid_0's rmse: 0.110854
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[244]	valid_0's rmse: 0.0469676
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[256]	valid_0's rmse: 0.519613
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[140]	valid_0's rmse: 0.472769
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[84]	valid_0's rmse: 0.601094
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[429]	valid_0's rmse: 0.0633792
Training until validation scores don't impr

In [ ]:
def create_lightgbm_submission(
    df_kaggle_test, lgbm_models, feature_cols,
    filename="submission_lgbm_per_store_item.csv"
):
    df_kaggle = df_kaggle_test.copy()
    kaggle_parts = []

    pairs_test = df_kaggle[['store_nbr', 'item_nbr']].drop_duplicates()

    for _, row in pairs_test.iterrows():
        s, i = int(row['store_nbr']), int(row['item_nbr'])
        key = (s, i)

        test_si = df_kaggle[
            (df_kaggle["store_nbr"] == s) &
            (df_kaggle["item_nbr"] == i)
        ].copy()
        if test_si.empty:
            continue

        X_test_si = test_si[feature_cols]

        if key in lgbm_models:
            yhat_log = lgbm_models[key].predict(X_test_si)
            units = np.expm1(yhat_log)
            units = np.clip(units, 0, None)
        else:
            units = np.zeros(len(test_si))

        test_si["units"] = units
        kaggle_parts.append(test_si)

    if not kaggle_parts:
        raise ValueError("Không có dòng nào được dự đoán.")

    df_kaggle_pred = pd.concat(kaggle_parts, ignore_index=True)

    df_kaggle_pred["date_str"] = df_kaggle_pred["date"].dt.strftime("%Y-%m-%d")
    df_kaggle_pred = df_kaggle_pred.sort_values(["date_str", "store_nbr", "item_nbr"])

    df_kaggle_pred["id"] = (
        df_kaggle_pred["store_nbr"].astype(str) + "_" +
        df_kaggle_pred["item_nbr"].astype(str) + "_" +
        df_kaggle_pred["date_str"]
    )

    submission = df_kaggle_pred[["id", "units"]].reset_index(drop=True)
    submission.to_csv(filename, index=False)
    print(f"✓ Saved {filename} ({len(submission)} rows)")
    return submission

submission_lgbm = create_lightgbm_submission(
    df_kaggle_test, lgbm_models, feature_cols,
    filename="submission_lgbm_per_store_item.csv"
)

✓ Saved submission_lgbm_cv_ensemble.csv (526917 rows)


In [14]:
# Kiểm tra nhanh: Đếm số lượng dòng dự đoán khác 0
non_zero_preds = submission_lgbm[submission_lgbm['units'] > 0]
print(f"Số lượng dòng có dự đoán bán hàng: {len(non_zero_preds)}")
print("Ví dụ 5 dòng có số liệu:")
print(non_zero_preds.head())

Số lượng dòng có dự đoán bán hàng: 24508
Ví dụ 5 dòng có số liệu:
                 id       units
4    2_5_2013-04-01   73.945860
10  2_11_2013-04-01    0.555558
43  2_44_2013-04-01  129.109535
84  2_85_2013-04-01    0.165281
92  2_93_2013-04-01    0.384099


### (Optional) Fine tunning using Optuna

In [ ]:
# def optimize_lightgbm(X_train, y_train, X_valid, y_valid, n_trials=50):
#     print("\nOptimizing LightGBM model with Optuna...")

#     def objective(trial):
#         # Hyperparameters search space
#         params = {
#             "objective": "regression",
#             "metric": "rmse", 
#             "boosting_type": "gbdt",
#             "verbosity": -1,
#             "num_leaves": trial.suggest_int("num_leaves", 31, 127),
#             "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
#             "feature_fraction": trial.suggest_float("feature_fraction", 0.7, 1.0),
#             "bagging_fraction": trial.suggest_float("bagging_fraction", 0.7, 1.0),
#             "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
#             "min_child_samples": trial.suggest_int("min_child_samples", 20, 100),
#             "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
#             "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
#             "max_depth": trial.suggest_int("max_depth", 5, 15),
#             "n_estimators": 2000,
#         }

#         model = lgbm.LGBMRegressor(**params)
#         # Train với early stopping
#         model.fit(
#             X_train, y_train,
#             eval_set=[(X_valid, y_valid)],
#             callbacks=[lgbm.early_stopping(stopping_rounds=100), lgbm.log_evaluation(0)],
#         )
        
#         # Predict và tính metric mục tiêu (WAPE)
#         preds = model.predict(X_valid)
#         wape = weighted_absolute_percentage_error(y_valid, preds)
#         return wape  # Optimize trực tiếp WAPE

#     # Chạy Optuna
#     study = optuna.create_study(direction="minimize")
#     study.optimize(objective, n_trials=n_trials)

#     print("\nBest params found:")
#     best_params = study.best_params
#     best_params.update({
#         "objective": "regression",
#         "metric": "rmse",
#         "boosting_type": "gbdt",
#         "verbosity": -1,
#         "n_estimators": 2000
#     })
    
#     for k, v in best_params.items():
#         print(f"  {k}: {v}")

#     # Train final model với best params + best iteration tìm được
#     final_model = lgbm.LGBMRegressor(**best_params)
#     final_model.fit(
#         X_train, y_train,
#         eval_set=[(X_valid, y_valid)],
#         callbacks=[lgbm.early_stopping(stopping_rounds=100), lgbm.log_evaluation(100)]
#     )
    
#     # Đánh giá lại
#     valid_preds = final_model.predict(X_valid)
#     test_mae = mean_absolute_error(y_valid, valid_preds)
#     test_rmse = np.sqrt(mean_squared_error(y_valid, valid_preds))
#     test_wape = weighted_absolute_percentage_error(y_valid, valid_preds)

#     print(f"\nOptimized LightGBM Valid Metrics - MAE: {test_mae:.3f}, RMSE: {test_rmse:.3f}, WAPE: {test_wape:.3f}")

#     return final_model, best_params, (test_mae, test_rmse, test_wape)

In [ ]:
# optimized_model, best_params, optimized_metrics = optimize_lightgbm(
#     X_train, y_train, X_valid, y_valid, n_trials=50
# )


[I 2025-11-29 04:23:49,946] A new study created in memory with name: no-name-c3196c8c-4586-46cc-80c0-051ed1084767



Optimizing LightGBM model with Optuna...
Training until validation scores don't improve for 100 rounds


[I 2025-11-29 04:23:55,842] Trial 0 finished with value: 14.823612109046277 and parameters: {'num_leaves': 108, 'learning_rate': 0.07028361509874961, 'feature_fraction': 0.8979200588036067, 'bagging_fraction': 0.9352906501638865, 'bagging_freq': 5, 'min_child_samples': 66, 'lambda_l1': 5.803801679413682e-07, 'lambda_l2': 1.2363809845200926, 'max_depth': 10}. Best is trial 0 with value: 14.823612109046277.


Early stopping, best iteration is:
[247]	valid_0's rmse: 0.363119
Training until validation scores don't improve for 100 rounds


[I 2025-11-29 04:24:31,672] Trial 1 finished with value: 14.726500982575583 and parameters: {'num_leaves': 80, 'learning_rate': 0.012204424379013217, 'feature_fraction': 0.8383414947848707, 'bagging_fraction': 0.7904500968473178, 'bagging_freq': 2, 'min_child_samples': 73, 'lambda_l1': 2.1459966988171709e-07, 'lambda_l2': 3.623957818166913e-08, 'max_depth': 13}. Best is trial 1 with value: 14.726500982575583.


...

Early stopping, best iteration is:
[559]	valid_0's rmse: 0.36141
Training until validation scores don't improve for 100 rounds


[I 2025-11-29 04:31:35,487] Trial 48 finished with value: 14.724125230565805 and parameters: {'num_leaves': 96, 'learning_rate': 0.0216736748580356, 'feature_fraction': 0.8090616951039101, 'bagging_fraction': 0.8114673953207123, 'bagging_freq': 4, 'min_child_samples': 60, 'lambda_l1': 1.589625974691346e-07, 'lambda_l2': 0.00041724139767755636, 'max_depth': 11}. Best is trial 35 with value: 14.675032358008016.


Early stopping, best iteration is:
[542]	valid_0's rmse: 0.361224
Training until validation scores don't improve for 100 rounds


[I 2025-11-29 04:31:42,595] Trial 49 finished with value: 14.710277199744219 and parameters: {'num_leaves': 102, 'learning_rate': 0.015149133328414935, 'feature_fraction': 0.7900585768548863, 'bagging_fraction': 0.9329870476567588, 'bagging_freq': 7, 'min_child_samples': 31, 'lambda_l1': 1.031868786034895e-05, 'lambda_l2': 0.00013172326777407766, 'max_depth': 9}. Best is trial 35 with value: 14.675032358008016.


Early stopping, best iteration is:
[830]	valid_0's rmse: 0.361216

Best params found:
  num_leaves: 127
  learning_rate: 0.012528590527215604
  feature_fraction: 0.8044026085648729
  bagging_fraction: 0.9028865865109351
  bagging_freq: 7
  min_child_samples: 42
  lambda_l1: 7.327783159306205e-08
  lambda_l2: 0.0035687388325977474
  max_depth: 11
  objective: regression
  metric: rmse
  boosting_type: gbdt
  verbosity: -1
  n_estimators: 2000
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 0.603716
[200]	valid_0's rmse: 0.389372
[300]	valid_0's rmse: 0.364696
[400]	valid_0's rmse: 0.361933
[500]	valid_0's rmse: 0.361409
[600]	valid_0's rmse: 0.361126
[700]	valid_0's rmse: 0.361064
[800]	valid_0's rmse: 0.360847
[900]	valid_0's rmse: 0.360956
Early stopping, best iteration is:
[850]	valid_0's rmse: 0.360807

Optimized LightGBM Valid Metrics - MAE: 0.200, RMSE: 0.361, WAPE: 14.675


In [ ]:
# submission_lgbm = create_lightgbm_submission(df_kaggle_test, optimized_model)

✓ Saved submission_lightgbm.csv (526917 rows)


## Evaluating model

In [ ]:
def evaluate_model(model, X_test, y_test, data):
    """
    Evaluate the model performance on the test set (last 3 months of 2017)
    """
    print("\nEvaluating model performance on test set...")

    # Make predictions on the test set
    test_preds = model.predict(X_test)

    # Calculate metrics
    test_mae = mean_absolute_error(y_test, test_preds)
    test_rmse = np.sqrt(mean_squared_error(y_test, test_preds))
    test_wape = weighted_absolute_percentage_error(y_test, test_preds)

    # Print evaluation results
    print(f"Final Model Test Evaluation:")
    print(f"    MAE: {test_mae:.2f}")
    print(f"    RMSE: {test_rmse:.2f}")
    print(f"    WAPE: {test_wape:.2f}%")

    # Analyze errors by time period (month)
    test_results = data[data["is_test"]].copy()
    test_results["prediction"] = test_preds
    test_results["error"] = test_results["sales"] - test_results["prediction"]
    test_results["abs_error"] = np.abs(test_results["error"])
    test_results["month_name"] = test_results["date"].dt.strftime("%B")

    # Summarize errors by month
    monthly_errors = (
        test_results.groupby("month_name")
        .agg({"abs_error": "mean", "error": "mean", "sales": "mean"})
        .reset_index()
    )
    monthly_errors["error_pct"] = (
        100 * monthly_errors["abs_error"] / monthly_errors["sales"]
    )

    print("\nError Analysis by Month:")
    print(
        monthly_errors[["month_name", "abs_error", "error_pct"]].to_string(index=False)
    )

    # Store results for visualization
    # Include month and store information for granular analysis
    test_results["year_month"] = test_results["date"].dt.strftime("%Y-%m")

    # Plot actual vs predicted
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, test_preds, alpha=0.5)
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], "r--")
    plt.title("Actual vs Predicted Sales (Test Set)")
    plt.xlabel("Actual Sales")
    plt.ylabel("Predicted Sales")
    plt.tight_layout()
    # plt.savefig('actual_vs_predicted_test.png')

    # Plot error distribution
    plt.figure(figsize=(10, 6))
    sns.histplot(test_results["error"], kde=True)
    plt.title("Error Distribution")
    plt.xlabel("Prediction Error")
    plt.tight_layout()
    # plt.savefig('error_distribution.png')

    return test_mae, test_rmse, test_wape, test_preds, y_test, test_results

In [ ]:
# Prophet Model Results:
# MAE: 9.03 | RMSE: 11.86 | WAPE: 29.13%

In [ ]:
# Evaluate the lightgbm model
test_mae, test_rmse, test_smape, test_preds, y_test_values, test_results = (
    evaluate_model(lightgbm_model, X_test, y_test, df_features)
)

NameError: name 'X_test' is not defined

## Save trained models

In [ ]:
def save_model(model, X_train, feature_names, output_dir="../models"):
    """
    Save the trained model and related artifacts for API use

    Args:
        model: Trained model (e.g., LightGBM model)
        feature_names: List of feature names
        output_dir: Directory to save model artifacts
    """

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Save the model
    model_path = os.path.join(output_dir, "sales_forecast_model.pkl")
    with open(model_path, "wb") as f:
        pickle.dump(model, f)
    print(f"Model saved to {model_path}")

    # Create and save feature statistics
    feature_stats = {
        "model_version": "1.0.0",
        "last_trained": pd.Timestamp.now().strftime("%Y-%m-%d"),
        "required_columns": list(feature_names),
        "column_order": list(feature_names),
        "default_values": {},
        "temperature_bins": [-np.inf, 20, 25, 30, np.inf],
        "temperature_labels": ["Cold", "Cool", "Warm", "Hot"],
        "humidity_bins": [-np.inf, 60, 75, np.inf],
        "humidity_labels": ["Low", "Medium", "High"],
    }

    # Add default values for date features
    feature_stats["default_values"] = {
        "year": 2017,
        "month": 11,
        "day": 15,
        "day_of_week": 2,
        "is_weekend": 0,
        "quarter": 4,
        "is_holiday": 0,
    }

    # Save feature stats
    stats_path = os.path.join(output_dir, "feature_stats.json")
    with open(stats_path, "w") as f:
        json.dump(feature_stats, f, indent=4)
    print(f"Feature statistics saved to {stats_path}")

    print(f"All model artifacts saved successfully to {output_dir}/")

    return model_path, stats_path

In [ ]:
# Save model
save_model(
    model=optimized_model,
    X_train=X_train,
    feature_names=X_train.columns,
    output_dir='../models'
)

Model saved to ../models/sales_forecast_model.pkl
Feature statistics saved to ../models/feature_stats.json
All model artifacts saved successfully to ../models/


('../models/sales_forecast_model.pkl', '../models/feature_stats.json')